# ***Estimation of GS and component biomass for forest sites which created 2020 and 2021 based on full data colection ***


# Load all data from .csv file

In [1]:
# Import main libraries

import numpy as np
import pandas as pd

In [2]:
# Load forest sites dataset
site_data = './output_other_authors_sites_biomass_20230125.xlsx'
df = pd.read_excel(site_data, sheet_name='output_2')
df.head()

,Groups,Species,Species_ID,Origin,Origin_ID,SI,A_years,H_m,DBH_cm,N_trees,...,Ln_xgb_allStem,Ln_xgb_stemBark,Ln_xgb_branch,Ln_xgb_foliage,T_ln_xgb_GS,T_ln_xgb_allStem,T_ln_xgb_stemBark,T_ln_xgb_branch,T_ln_xgb_foliage,T_ln_xgb_Ab_g
0,1,Alder,3,a,1.0,2,33,16.0,12.4,1700.0,...,4.326847,2.392613,1.930532,0.718511,170.774933,75.705215,10.942053,6.893176,2.051377,84.649765
1,1,Alder,3,a,1.0,-2,42,22.9,19.4,1136.0,...,5.154976,3.195204,1.898181,0.612538,398.806671,173.291626,24.415154,6.673744,1.845109,181.810471
2,2,Alder,3,a,1.0,2,33,16.0,12.4,1700.0,...,4.326847,2.392613,1.930532,0.718511,170.774933,75.705215,10.942053,6.893176,2.051377,84.649765
3,2,Alder,3,a,1.0,0,43,21.0,20.8,660.0,...,4.663519,2.764480,2.172356,0.841122,227.961624,106.008514,15.870779,8.778940,2.318966,117.106422
4,2,Alder,3,a,1.0,0,28,15.4,14.2,1369.0,...,4.341617,2.434403,2.056040,0.678623,169.117905,76.831680,11.409010,7.814958,1.971162,86.617798


In [3]:
# Information on all columns of data frame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857 entries, 0 to 856
Data columns (total 46 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Groups             857 non-null    int64  
 1   Species            857 non-null    object 
 2   Species_ID         857 non-null    int64  
 3   Origin             856 non-null    object 
 4   Origin_ID          856 non-null    float64
 5   SI                 857 non-null    int64  
 6   A_years            857 non-null    int64  
 7   H_m                857 non-null    float64
 8   DBH_cm             856 non-null    float64
 9   N_trees            846 non-null    float64
 10  RS(P)              854 non-null    float64
 11  BA_sq_m_sq_m       857 non-null    float64
 12  GS_cub_m           857 non-null    float64
 13  ALL_STEM_t_ha      855 non-null    float64
 14  Stem_wood_t_ha     747 non-null    float64
 15  Stem_bark_t_ha     747 non-null    float64
 16  Crown_t_ha         843 non

In [4]:
# Selecting main working parameters

select = pd.DataFrame(df, columns= ['Species_ID', 'Origin_ID', 'H_m', 'DBH_cm', 'BA_sq_m_sq_m'])
'''
In the "ID_species" column: "0" is Pine, "1" is Spruce, "2" is Birch, "3" is Alder, 
"4" is Aspen, "5" is Oak
In the "ID_origin" column: "0" is nutural stand, "1" is artificial stand
'''

# Show the first five columns
select.head()

,Species_ID,Origin_ID,H_m,DBH_cm,BA_sq_m_sq_m
0,3,1.0,16.0,12.4,20.6
1,3,1.0,22.9,19.4,33.5
2,3,1.0,16.0,12.4,20.6
3,3,1.0,21.0,20.8,22.4
4,3,1.0,15.4,14.2,21.7


# Review of input data for classification

In [5]:
# Information on all columns of data frame
select.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857 entries, 0 to 856
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Species_ID    857 non-null    int64  
 1   Origin_ID     856 non-null    float64
 2   H_m           857 non-null    float64
 3   DBH_cm        856 non-null    float64
 4   BA_sq_m_sq_m  857 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 33.6 KB


In [6]:
# Descriptive statistics of main parameters
select.describe()

,Species_ID,Origin_ID,H_m,DBH_cm,BA_sq_m_sq_m
count,857.000000,856.000000,857.000000,856.000000,857.000000
mean,1.529755,0.279206,16.452275,17.132243,26.329988
std,1.823872,0.448871,7.039954,10.439115,10.629993
min,0.000000,0.000000,1.500000,1.100000,1.000000
25%,0.000000,0.000000,11.200000,9.900000,19.000000
50%,0.000000,0.000000,16.100000,15.500000,26.800000
75%,3.000000,1.000000,21.200000,22.100000,34.500000
max,5.000000,1.000000,38.300000,94.000000,51.500000


# Selecting working columns for the XGBoost algorithms

In [7]:
# We select independent variables and predicting parameter 
X = select


In [8]:
# Columns in "X" data collections
X.head()

,Species_ID,Origin_ID,H_m,DBH_cm,BA_sq_m_sq_m
0,3,1.0,16.0,12.4,20.6
1,3,1.0,22.9,19.4,33.5
2,3,1.0,16.0,12.4,20.6
3,3,1.0,21.0,20.8,22.4
4,3,1.0,15.4,14.2,21.7


# Load XGBoost module and learned model for predict of species and other lands

In [9]:
#  Importing the main library for building model and its analysis

import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [10]:
# Load learned models

xgb_model_GS = xgb.XGBRegressor()
xgb_model_allStem = xgb.XGBRegressor()
xgb_model_stemBark = xgb.XGBRegressor()
xgb_model_br = xgb.XGBRegressor()
xgb_model_f = xgb.XGBRegressor()
xgb_model_LN_GS = xgb.XGBRegressor()
xgb_model_LN_allStem = xgb.XGBRegressor()
xgb_model_LN_stemBark = xgb.XGBRegressor()
xgb_model_LN_br = xgb.XGBRegressor()
xgb_model_LN_f = xgb.XGBRegressor()
xgb_model_GS.load_model('./XGBoost_models/01_CS_other_authors_XGB_model.json')
xgb_model_allStem.load_model('./XGBoost_models/02_M_all_stem_other_authors_XGB_model.json')
xgb_model_stemBark.load_model('./XGBoost_models/03_M_stem_bark_other_authors_XGB_model.json')
xgb_model_br.load_model('./XGBoost_models/04_M_branch_other_authors_XGB_model.json')
xgb_model_f.load_model('./XGBoost_models/05_M_foliage_other_authors_XGB_model.json')
xgb_model_LN_GS.load_model('./XGBoost_models/01_LN_CS_other_authors_XGB_model.json')
xgb_model_LN_allStem.load_model('./XGBoost_models/02_LN_M_all_stem_other_authors_XGB_model.json')
xgb_model_LN_stemBark.load_model('./XGBoost_models/03_LN_M_stem_bark_other_authors_XGB_model.json')
xgb_model_LN_br.load_model('./XGBoost_models/04_LN_M_branch_other_authors_XGB_model.json')
xgb_model_LN_f.load_model('./XGBoost_models/05_LN_M_foliage_other_authors_XGB_model.json')

In [11]:
# Show all parameters of XGBoost models

print(xgb_model_GS)
print(xgb_model_allStem)
print(xgb_model_stemBark)
print(xgb_model_br)
print(xgb_model_f)
print(xgb_model_LN_GS)
print(xgb_model_LN_allStem)
print(xgb_model_LN_stemBark)
print(xgb_model_LN_br)
print(xgb_model_LN_f)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.8829191872975016, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=['rmse'],
             gamma=0.4317747333990657, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.17029218845451208, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=4, max_leaves=0,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=172, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0, reg_lambda=1, ...)
XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.7658578011184654, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=['rmse

In [14]:
# Create predict values

pred_GS = xgb_model_GS.predict(X, ntree_limit=xgb_model_GS.best_ntree_limit)
pred_allStem = xgb_model_allStem.predict(X, ntree_limit=xgb_model_allStem.best_ntree_limit)
pred_stemBark = xgb_model_stemBark.predict(X, ntree_limit=xgb_model_stemBark.best_ntree_limit)
pred_br = xgb_model_br.predict(X, ntree_limit=xgb_model_br.best_ntree_limit)
pred_f = xgb_model_f.predict(X, ntree_limit=xgb_model_f.best_ntree_limit)
pred_LN_GS = xgb_model_LN_GS.predict(X, ntree_limit=xgb_model_LN_GS.best_ntree_limit)
pred_LN_allStem = xgb_model_LN_allStem.predict(X, ntree_limit=xgb_model_LN_allStem.best_ntree_limit)
pred_LN_stemBark = xgb_model_LN_stemBark.predict(X, ntree_limit=xgb_model_LN_stemBark.best_ntree_limit)
pred_LN_br = xgb_model_LN_br.predict(X, ntree_limit=xgb_model_LN_br.best_ntree_limit)
pred_LN_f = xgb_model_LN_f.predict(X, ntree_limit=xgb_model_LN_f.best_ntree_limit)

In [15]:
# Convert predicted parameters to real values

R_pred_GS = np.exp(pred_LN_GS)
R_pred_allStem = np.exp(pred_LN_allStem)
R_pred_stemBark = np.exp(pred_LN_stemBark)
R_pred_br = np.exp(pred_LN_br)
R_pred_f = np.exp(pred_LN_f)

In [16]:
data = {'xgb_GS_ver2':pred_GS, 'xgb_allStem_ver2':pred_allStem, 'xgb_stemBark_ver2':pred_stemBark, 'xgb_branch_ver2':pred_br, 'xgb_foliage_ver2':pred_f, 'T_ln_xgb_GS_ver2':R_pred_GS, 'T_ln_xgb_allStem_ver2':R_pred_allStem, 'T_ln_xgb_stemBark_ver2':R_pred_stemBark, 'T_ln_xgb_branch_ver2':R_pred_br, 'T_ln_xgb_foliage_ver2':R_pred_f}

In [17]:
# Add predicted values to the working dataframe
pred = pd.DataFrame(data)

In [18]:
# Show output data of biomass components
print(pred)

     xgb_GS_ver2  xgb_allStem_ver2  xgb_stemBark_ver2  xgb_branch_ver2  \
0     170.119598         76.865341          10.471112         7.064976   
1     423.108398        186.462311          21.708059         2.872464   
2     170.119598         76.865341          10.471112         7.064976   
3     241.354172        111.992638          16.712152         9.586866   
4     161.839508         74.577042          11.710971         7.555252   
..           ...               ...                ...              ...   
852   182.234726         70.456841          19.261515        30.230337   
853   283.821442        126.276100          31.591810        41.531120   
854   362.459869        151.860016          45.395206        50.003254   
855   421.191284        191.152466          57.074516        52.018570   
856   444.747925        212.099380          65.154579        56.373936   

     xgb_foliage_ver2  T_ln_xgb_GS_ver2  T_ln_xgb_allStem_ver2  \
0            2.508739        169.713287      

In [19]:
# Show description statistics

pred.describe()

,xgb_GS_ver2,xgb_allStem_ver2,xgb_stemBark_ver2,xgb_branch_ver2,xgb_foliage_ver2,T_ln_xgb_GS_ver2,T_ln_xgb_allStem_ver2,T_ln_xgb_stemBark_ver2,T_ln_xgb_branch_ver2,T_ln_xgb_foliage_ver2
count,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000
mean,229.032349,102.690201,11.733580,13.750736,4.753818,228.497559,101.704918,11.343656,13.095243,4.639531
std,143.711197,68.096649,9.055766,11.144939,2.352741,142.958206,66.879623,8.744547,10.862898,2.522044
min,6.038515,2.876050,0.659912,1.250499,0.558210,2.713782,2.281078,0.425623,0.719618,0.220431
25%,115.901009,48.536453,6.345463,7.064976,2.893557,114.897285,48.323376,6.085863,6.793105,2.678547
50%,207.456726,94.577545,8.919965,10.980662,4.559431,207.668533,92.755615,8.958900,10.480473,4.360866
75%,320.528931,139.478073,14.770077,15.685203,6.335762,319.011261,140.487289,13.481342,14.646773,6.209807
max,734.102783,363.314209,65.154579,69.117683,19.017195,737.108826,351.203979,49.763363,68.178345,18.455608


In [20]:
# Add predicted values to the working dataframe

df = df.join(pred)

In [21]:
# Calculating the sum of above-ground biomass using the XGBoost algorithm

sum_biomass = df['xgb_allStem_ver2'] + df['xgb_branch_ver2'] + df['xgb_foliage_ver2']
df['xgb_Ab_g_ver2'] = sum_biomass

ln_sum_biomass = df['T_ln_xgb_allStem_ver2'] + df['T_ln_xgb_branch_ver2'] + df['T_ln_xgb_foliage_ver2']
df['T_ln_xgb_Ab_g_ver2'] = ln_sum_biomass

In [22]:
# Save new dataframe with predict values to .xlsx-file

df.to_excel('./Est_biomass_use_final_models_10052023.xlsx', sheet_name='output', index=False)